In [5]:
import math
import os
import janitor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from dotenv import load_dotenv
from google.cloud import bigquery
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, max_error
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, RobustScaler, StandardScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, LSTM, LeakyReLU
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
load_dotenv()
import os

# my functions
from mymodule import create_csv_file
from mymodule import remove_pattern
from mymodule import trim_to_three_cd10
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/amermujkanovic/code/Amerigo23/MaximonHealth/gcp.json'


In [38]:
# columns to create the diseases columsn
diesaes_columns= ['pdx', 'dx1','dx2','dx3','dx4','dx5','dx6','dx7','dx8','dx9','dx10','dx11','dx12','dx13','dx14','dx15']

# 100 most freqeunt diseases dicationary to create one hot encoding
disease_icd10_dict = {
    "Acute Myocardial Infarction (Heart Attack)": "I21",
    "Angina Pectoris": "I20",
    "Aortic Aneurysm": "I71",
    "Peripheral Artery Disease": "I73",
    "Thrombophlebitis": "I80",
    "Varicose Veins": "I83",
    "Heart Failure": "I50",
    "Atrial Fibrillation": "I48",
    "Cardiomyopathy": "I42",
    "Mitral Valve Disorders": "I34",
    "Rheumatic Heart Disease": "I01",
    "Hypertensive Heart Disease": "I11",
    "Venous Insufficiency": "I87.2",
    "Raynaud's Syndrome": "I73.0",
    "Acute Bronchitis": "J20",
    "Chronic Bronchitis": "J42",
    "Pneumonia": "J18",
    "Pulmonary Edema": "J81",
    "Pulmonary Embolism": "I26",
    "Pleurisy": "R09.1",
    "Asthma": "J45",
    "Emphysema": "J43",
    "Sleep Apnea": "G47.3",
    "Lung Cancer": "C34",
    "Interstitial Lung Disease": "J84",
    "Sarcoidosis": "D86",
    "Cystic Fibrosis": "E84",
    "Pulmonary Hypertension": "I27",
    "Tuberculosis (TB)": "A15",
    "Silicosis": "J62",
    "Acute Upper Respiratory Infections": "J06",
    "Influenza": "J09-J11",
    "Tonsillitis": "J03",
    "Pharyngitis": "J02",
    "Laryngitis": "J04",
    "Sinusitis": "J01",
    "Otitis Media": "H66",
    "Meniere's Disease": "H81",
    "Hearing Loss": "H90",
    "Vertigo": "R42",
    "Tinnitus": "H93",
    "Conjunctivitis": "H10",
    "Cataract": "H25",
    "Glaucoma": "H40",
    "Diabetic Retinopathy": "E14",
    "Macular Degeneration": "H35",
    "Refractive Errors": "H52",
    "Strabismus": "H50",
    "Corneal Ulcer": "H16",
    "Dry Eye Syndrome": "H04",
    "Retinal Detachment": "H33",
    "Uveitis": "H20",
    "Keratitis": "H16",
    "Blepharitis": "H01",
    "Stye (Hordeolum)": "H00",
    "Chalazion": "H00",
    "Acanthamoeba Keratitis": "B60",
    "Trachoma": "A71",
    "Acute Appendicitis": "K35",
    "Cholecystitis": "K81",
    "Pancreatitis": "K85",
    "Hepatitis A": "B15",
    "Hepatitis B": "B16",
    "Hepatitis C": "B17",
    "Alcoholic Liver Disease": "K70",
    "Non-Alcoholic Fatty Liver Disease": "K76.0",
    "Cirrhosis": "K74",
    "Gastric Ulcer": "K25",
    "Duodenal Ulcer": "K26",
    "Gastroenteritis": "A09",
    "Diverticulitis": "K57",
    "Irritable Bowel Syndrome": "K58",
    "Inflammatory Bowel Disease": "K50",
    "Inflammatory Bowel Disease": "K51",
    "Celiac Disease": "K90.0",
    "Lactose Intolerance": "E73.9",
    "Intestinal Obstruction": "K56",
    "Hemorrhoids": "K64",
    "Anal Fissure": "K60",
    "Rectal Prolapse": "K62",
    "Esophageal Varices": "I85",
    "Gastroesophageal Reflux Disease (GERD)": "K21",
    "Peptic Ulcer Disease": "K27",
    "Chronic Kidney Disease": "N18",
    "Nephrolithiasis (Kidney Stones)": "N20",
    "Acute Kidney Injury": "N17",
    "Glomerulonephritis": "N00-N08",
    "Urinary Tract Infection": "N39",
    "Bladder Cancer": "C67",
    "Prostate Cancer": "C61",
    "Benign Prostatic Hyperplasia": "N40",
    "Erectile Dysfunction": "N52",
    "Urinary Incontinence": "N39",
    "Interstitial Cystitis": "N30",
    "Polycystic Kidney Disease": "Q61",
    "Acute Cystitis": "N30.0",
    "Pyelonephritis": "N10",
    "Urethritis": "N34",
    "Male Infertility": "N46",
    "Female Infertility": "N97",
    "Polycystic Ovary Syndrome (PCOS)": "E28"
}
diesaes_columns_names = list(diesaes_columns)

#region codes
state_codes = {
 1: 'Nation, unknown region',
 2: 'Northeast Region, unknown division',
 3: 'New England Division, unknown state',
 4: 'Connecticut',
 5: 'Maine',
 6: 'Massachusetts',
 7: 'New Hampshire',
 8: 'Rhode Island',
 9: 'Vermont',
 10: 'Middle Atlantic Division, unknown state',
 11: 'New Jersey',
 12: 'New York',
 13: 'Pennsylvania',
 14: 'North Central Region, unknown division',
 15: 'East North Central Division, unknown state',
 16: 'Illinois',
 17: 'Indiana',
 18: 'Michigan',
 19: 'Ohio',
 20: 'Wisconsin',
 21: 'West North Central Division, unknown state',
 22: 'Iowa',
 23: 'Kansas',
 24: 'Minnesota',
 25: 'Missouri',
 26: 'Nebraska',
 27: 'North Dakota',
 28: 'South Dakota',
 29: 'South Region, unknown division',
 30: 'South Atlantic Division, unknown state',
 31: 'Washington, DC',
 32: 'Delaware',
 33: 'Florida',
 34: 'Georgia',
 35: 'Maryland',
 36: 'North Carolina',
 37: 'South Carolina',
 38: 'Virginia',
 39: 'West Virginia',
 40: 'East South Central Division, unknown state',
 41: 'Alabama',
 42: 'Kentucky',
 43: 'Mississippi',
 44: 'Tennessee',
 45: 'West South Central Division, unknown state',
 46: 'Arkansas',
 47: 'Louisiana',
 48: 'Oklahoma',
 49: 'Texas',
 50: 'West Region, unknown division',
 51: 'Mountain Division, unknown state',
 52: 'Arizona',
 53: 'Colorado',
 54: 'Idaho',
 55: 'Montana',
 56: 'Nevada',
 57: 'New Mexico',
 58: 'Utah',
 59: 'Wyoming',
 60: 'Pacific Division, unknown state',
 61: 'Alaska',
 62: 'California',
 63: 'Hawaii',
 64: 'Oregon',
 65: 'Washington',
 97: 'Puerto Rico'}
state_column_names = list(state_codes.values())

#discharge codees
sorted_discharge_status = {
    1: 'Discharged to home or self-care',
    2: 'Discharged/Transferred to short-term hospital',
    3: 'Discharged/Transferred to SNF',
    4: 'Discharged/Transferred to ICF',
    5: 'Discharged/Transferred to other facility',
    6: 'Discharged/Transferred to home health service',
    7: 'Left against medical advice',
    8: 'Discharged/Transferred to home IV drug therapy',
    9: 'Admitted as an inpatient to this hospital',
    10: 'Other alive status',
    11: 'Other alive status',
    12: 'Other alive status',
    13: 'Other alive status',
    14: 'Other alive status',
    15: 'Other alive status',
    16: 'Other alive status',
    17: 'Other alive status',
    18: 'Other alive status',
    19: 'Other alive status',
    20: 'Died**',
    21: 'Discharged/transferred to court/law enforcement**',
    30: 'Still patient',
    31: 'Still patient',
    32: 'Still patient',
    33: 'Still patient',
    34: 'Still patient',
    35: 'Still patient',
    36: 'Still patient',
    37: 'Still patient',
    38: 'Still patient',
    39: 'Still patient',
    41: 'Expired in medical facility (Hospice claims only)**',
    42: 'Expired - place unknown (Hospice claims only)**',
    43: 'Discharged/Transferred to federal hospital',
    50: 'Discharged/Transferred to Hospice home',
    51: 'Discharged/Transferred to Hospice medical facility',
    61: 'Transfer to Medicare approved swing-bed',
    62: 'Transferred to inpatient rehab facility (IRF)',
    63: 'Transferred to long term care hospital (LTCH)',
    64: 'Transferred to nursing facility Medicaid-certified',
    65: 'Transferred to psychiatric hospital or unit',
    66: 'Transferred to critical access hospital (CAH)',
    69: 'Transfer to disaster alternative care site',
    70: 'Transfer to another facility NEC',
    71: 'Transfer/referred to other facility for outpt svcs',
    72: 'Transfer/referred to this facility for outpt svcs',
    81: 'Discharge to home/self care w plan IP readmit',
    82: 'Transfer to short-term general hosp w/ plan IP readmit',
    83: 'Transfer to SNF w/ plan IP readmit',
    84: 'Transfer to custodial/supportive care w/ plan IP readmit',
    85: 'Transfer to cancer center/child hosp w/ plan IP readmit',
    86: 'Transfer to home health service w/ plan IP readmit',
    87: 'Transfer to court/law enforce w/ plan IP readmit**',
    88: 'Transfer to federal facility HCF w/ plan IP readmit',
    89: 'Transfer to Medicare swing bed w/ plan IP readmit',
    90: 'Transfer to IRF w/ plan IP readmit',
    91: 'Transfer to LTCH w/ plan IP readmit',
    92: 'Transfer to Medicaid nursing facility w/ plan IP readmit',
    93: 'Transfer to psych unit/hospital w/ plan IP readmit',
    94: 'Transfer to CAH w/ plan IP readmit',
    95: 'Transfer to other facility NEC w/ plan IP readmit',
    99: 'Transfer (Hospital ID MDST change)'
}


In [ ]:
google_credentials = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
PROJECT = "focus-hulling-416322"
DATASET = "2016"
TABLE = "ccaei163_narrow"

# Ensure the project, dataset, and table names are correctly formatted
# using backticks to avoid syntax errors
query = f"""
    SELECT *
    FROM `{PROJECT}.{DATASET}.{TABLE}`
"""

client = bigquery.Client(project=PROJECT)
query_job = client.query(query)
result = query_job.result()
data = result.to_dataframe()
data = data.applymap(remove_pattern)


In [39]:
df = data.clean_names()
trim_to_three_cd10(df, diseases_columns)

for disease_name, icd_code in disease_icd10_dict.items():
    df[disease_name] = df.apply(lambda row: any(row[col] == icd_code for col in diseases_columns), axis=1).astype(int)

# Drop the disease code columns after processing
df = df.drop(diseases_columns, axis=1)

# Process region codes
for code, state_name in state_codes.items():
    df[state_name] = (df['state'] == code).astype(int)

# Drop the region_code column after processing
df = df.drop('state', axis=1)


In [18]:
df

,efamid,enrolid,eidflag,physid,caseid,age,sex,region,state,days,drg,disdate,hospnet,hosppay,mhsacovg,pdx,dx1,dx2,dx3,dx4,dx5,dx6,dx7,dx8,dx9,dx10,dx11,dx12,dx13,dx14,dx15,dstatus,physnet,physpay,plantyp,hlthplan,pproc,eeclass,eestatu,egeoloc,emprel,rx,totcob,totcoins,totcopay,totded,totnet,totpay
0,39581991,3958199102,1,29842521.0,1346649,61,2,5,49,5,389,2016-02-05,15329.51,15329.51,1,K56,K56,G89,N13,R05,R10,K59,C20,K31,R06,R11,C79,R64,R68,Non,Non,01,689.45,689.45,2.0,0,4,5,1,01,2,1,0.0,0.0,0.0,0.00,17388.85,17388.85
1,39601661,3960166101,1,118615807.0,1348901,60,2,5,49,15,7,2016-10-27,206820.26,206820.26,1,J84,J84,I34,R07,R07,J84,J84,Z94,R73,Z76,M25,I36,J18,J96,I10,I51,06,22367.44,22795.91,6.0,0,4,1,9,01,1,1,0.0,0.0,0.0,428.47,229718.40,230146.87
2,297791054,29779105402,1,NaN,245966,51,2,5,06,3,377,2016-09-10,34863.04,24168.92,1,K92,K92,D50,D62,E83,E87,E87,F10,F32,G93,I49,Z45,D50,K21,K44,K52,07,1248.60,1248.60,6.0,1,None,9,9,01,2,1,0.0,0.0,600.0,0.00,36733.48,26039.36
3,332969309,33296930902,1,462658955.0,1445011,54,2,5,33,5,92,2016-09-21,9518.40,11898.00,1,R26,R26,D64,F10,F12,F17,G89,M48,M54,R25,M62,M79,M54,R26,Non,Non,01,280.13,280.13,6.0,1,None,9,1,01,2,1,0.0,2379.6,0.0,0.00,9887.48,12267.08
4,333683659,33368365901,1,658800401.0,1479311,54,1,5,64,1,392,2016-08-02,10108.40,10108.40,1,K31,K31,D68,E87,E87,E87,I50,K20,K25,M06,N28,R10,R91,R11,R07,R19,01,435.95,435.95,6.0,1,None,9,1,01,1,1,0.0,0.0,0.0,0.00,10987.88,10987.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363293,7611567,761156702,1,NaN,413370,52,1,5,97,7,689,2016-09-21,5027.00,5127.00,1,N39,N39,A41,N40,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,None,563.00,563.00,6.0,0,3,1,1,97,2,1,0.0,0.0,0.0,0.00,5827.00,5927.00
363294,7612423,761242302,1,NaN,413405,43,2,5,97,3,743,2016-03-14,3551.00,3651.00,1,D25,D25,N85,N92,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,None,740.00,740.00,6.0,0,3,4,1,97,2,1,0.0,0.0,0.0,0.00,4862.50,4962.50
363295,9217909,921790902,1,NaN,413672,57,2,5,97,2,694,2016-02-19,2544.00,2744.00,1,N13,N13,N20,I10,N20,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,None,598.00,598.00,6.0,0,3,1,1,97,2,1,0.0,0.0,0.0,0.00,3369.00,3569.00
363296,9595136,959513601,1,NaN,413695,55,2,5,97,10,872,2016-11-24,6480.00,6580.00,1,A41,A41,J45,J45,J45,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,None,302.00,302.00,6.0,0,3,4,1,97,1,1,0.0,9.0,0.0,0.00,6864.00,6973.00


In [10]:
duplicates = df.duplicated('enrolid',  keep=False)
count_duplicates = duplicates.sum()
count_duplicates

168259

In [13]:
duplicate_counts = df[df.duplicated('enrolid', keep=False)]['enrolid'].value_counts()
print(duplicate_counts)

enrolid
32702962003    29
2859418406     25
1059540402     23
3592415004     23
28468331003    23
               ..
3557164001      2
2675731802      2
1260407102      2
1269669801      2
1038904001      2
Name: count, Length: 61475, dtype: int64


In [19]:
pd.set_option('display.max_columns', None)
df[df['enrolid'] == 32702962003 ]

,efamid,enrolid,eidflag,physid,caseid,age,sex,region,state,days,drg,disdate,hospnet,hosppay,mhsacovg,pdx,dx1,dx2,dx3,dx4,dx5,dx6,dx7,dx8,dx9,dx10,dx11,dx12,dx13,dx14,dx15,dstatus,physnet,physpay,plantyp,hlthplan,pproc,eeclass,eestatu,egeoloc,emprel,rx,totcob,totcoins,totcopay,totded,totnet,totpay
10653,327029620,32702962003,1,NaN,71260,20,1,1,62,4,897,2016-10-15,7727.18,7727.18,1,F10,F10,F11,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,05,0.00,0.00,6.0,1,None,9,9,07,3,1,0.0,0.00,0.0,0.0,7727.18,7727.18
10695,327029620,32702962003,1,NaN,71264,20,1,1,62,10,897,2016-11-29,27218.00,27218.00,1,F10,F10,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,30,0.00,0.00,6.0,1,None,9,9,07,3,1,0.0,0.00,0.0,0.0,27218.00,27218.00
10723,327029620,32702962003,1,NaN,71252,20,1,1,62,2,897,2016-08-26,4201.50,4201.50,1,F13,F13,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,30,0.00,0.00,6.0,1,None,9,9,07,3,1,0.0,0.00,0.0,0.0,4201.50,4201.50
10724,327029620,32702962003,1,NaN,71254,20,1,1,62,1,897,2016-08-29,1300.50,1300.50,1,F13,F13,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,30,0.00,0.00,6.0,1,None,9,9,07,3,1,0.0,0.00,0.0,0.0,1300.50,1300.50
10725,327029620,32702962003,1,NaN,71256,20,1,1,62,3,894,2016-09-05,16259.60,16259.60,1,F11,F11,F11,F33,Z11,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,07,580.01,580.01,6.0,1,None,9,9,07,3,1,0.0,0.00,0.0,0.0,17050.45,17050.45
10726,327029620,32702962003,1,NaN,71258,20,1,1,62,1,897,2016-09-16,1357.22,1357.22,1,F11,F11,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,50,0.00,0.00,6.0,1,None,9,9,07,3,1,0.0,0.00,0.0,0.0,1357.22,1357.22
10788,327029620,32702962003,1,NaN,71257,20,1,1,62,5,897,2016-09-11,10494.00,10494.00,1,F13,F13,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,30,0.00,0.00,6.0,1,None,9,9,07,3,1,0.0,0.00,0.0,0.0,10494.00,10494.00
10803,327029620,32702962003,1,NaN,71266,20,1,1,06,3,897,2016-12-31,5566.00,5566.00,1,F10,F10,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,30,0.00,0.00,6.0,1,None,9,9,07,3,1,0.0,0.00,0.0,0.0,5566.00,5566.00
10815,327029620,32702962003,1,NaN,71241,20,1,1,62,9,897,2016-05-06,11161.25,11320.19,1,F13,F13,F19,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,30,946.00,946.00,6.0,1,None,9,9,07,3,1,0.0,158.93,0.0,0.0,12107.25,12266.19
10816,327029620,32702962003,1,NaN,71248,20,1,1,62,2,897,2016-08-02,10758.00,10758.00,1,F13,F13,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,30,0.00,0.00,6.0,1,None,9,9,07,3,1,0.0,0.00,0.0,0.0,10758.00,10758.00


In [24]:
grouped_df = df.groupby('group_column').agg({
    'sum_column1': 'sum',
    'sum_column2': 'sum',
    'mean_column1': 'mean',
    'mean_column2': 'mean'
    # Add more columns with their aggregation methods
})


/var/folders/fh/fc20820s23gdqf7bkj31z2rw0000gn/T/ipykernel_55577/2077537071.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[disease_name] = df.apply(lambda row: any (row[col] == icd_code for col in ['pdx', 'dx1','dx2','dx3','dx4','dx5','dx6','dx7','dx8','dx9','dx10','dx11','dx12','dx13','dx14','dx15']), axis=1).astype(int)
/var/folders/fh/fc20820s23gdqf7bkj31z2rw0000gn/T/ipykernel_55577/2077537071.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[disease_name] = df.apply(lambda row: any (row[col] == icd_code for col 

,efamid,enrolid,eidflag,physid,caseid,age,sex,region,state,days,drg,disdate,hospnet,hosppay,mhsacovg,pdx,dx1,dx2,dx3,dx4,dx5,dx6,dx7,dx8,dx9,dx10,dx11,dx12,dx13,dx14,dx15,dstatus,physnet,physpay,plantyp,hlthplan,pproc,eeclass,eestatu,egeoloc,emprel,rx,totcob,totcoins,totcopay,totded,totnet,totpay,Acute Myocardial Infarction (Heart Attack),Angina Pectoris,Aortic Aneurysm,Peripheral Artery Disease,Thrombophlebitis,Varicose Veins,Heart Failure,Atrial Fibrillation,Cardiomyopathy,Mitral Valve Disorders,Rheumatic Heart Disease,Hypertensive Heart Disease,Venous Insufficiency,Raynaud's Syndrome,Acute Bronchitis,Chronic Bronchitis,Pneumonia,Pulmonary Edema,Pulmonary Embolism,Pleurisy,Asthma,Emphysema,Sleep Apnea,Lung Cancer,Interstitial Lung Disease,Sarcoidosis,Cystic Fibrosis,Pulmonary Hypertension,Tuberculosis (TB),Silicosis,Acute Upper Respiratory Infections,Influenza,Tonsillitis,Pharyngitis,Laryngitis,Sinusitis,Otitis Media,Meniere's Disease,Hearing Loss,Vertigo,Tinnitus,Conjunctivitis,Cataract,Glaucoma,Diabetic Retinopathy,Macular Degeneration,Refractive Errors,Strabismus,Corneal Ulcer,Dry Eye Syndrome,Retinal Detachment,Uveitis,Keratitis,Blepharitis,Stye (Hordeolum),Chalazion,Acanthamoeba Keratitis,Trachoma,Acute Appendicitis,Cholecystitis,Pancreatitis,Hepatitis A,Hepatitis B,Hepatitis C,Alcoholic Liver Disease,Non-Alcoholic Fatty Liver Disease,Cirrhosis,Gastric Ulcer,Duodenal Ulcer,Gastroenteritis,Diverticulitis,Irritable Bowel Syndrome,Inflammatory Bowel Disease,Celiac Disease,Lactose Intolerance,Intestinal Obstruction,Hemorrhoids,Anal Fissure,Rectal Prolapse,Esophageal Varices,Gastroesophageal Reflux Disease (GERD),Peptic Ulcer Disease,Chronic Kidney Disease,Nephrolithiasis (Kidney Stones),Acute Kidney Injury,Glomerulonephritis,Urinary Tract Infection,Bladder Cancer,Prostate Cancer,Benign Prostatic Hyperplasia,Erectile Dysfunction,Urinary Incontinence,Interstitial Cystitis,Polycystic Kidney Disease,Acute Cystitis,Pyelonephritis,Urethritis,Male Infertility,Female Infertility,Polycystic Ovary Syndrome (PCOS)
0,39581991,3958199102,1,29842521.0,1346649,61,2,5,49,5,389,2016-02-05,15329.51,15329.51,1,K56,K56,G89,N13,R05,R10,K59,C20,K31,R06,R11,C79,R64,R68,Non,Non,01,689.45,689.45,2.0,0,4,5,1,01,2,1,0.0,0.0,0.0,0.00,17388.85,17388.85,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,39601661,3960166101,1,118615807.0,1348901,60,2,5,49,15,7,2016-10-27,206820.26,206820.26,1,J84,J84,I34,R07,R07,J84,J84,Z94,R73,Z76,M25,I36,J18,J96,I10,I51,06,22367.44,22795.91,6.0,0,4,1,9,01,1,1,0.0,0.0,0.0,428.47,229718.40,230146.87,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,297791054,29779105402,1,NaN,245966,51,2,5,06,3,377,2016-09-10,34863.04,24168.92,1,K92,K92,D50,D62,E83,E87,E87,F10,F32,G93,I49,Z45,D50,K21,K44,K52,07,1248.60,1248.60,6.0,1,None,9,9,01,2,1,0.0,0.0,600.0,0.00,36733.48,26039.36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,332969309,33296930902,1,462658955.0,1445011,54,2,5,33,5,92,2016-09-21,9518.40,11898.00,1,R26,R26,D64,F10,F12,F17,G89,M48,M54,R25,M62,M79,M54,R26,Non,Non,01,280.13,280.13,6.0,1,None,9,1,01,2,1,0.0,2379.6,0.0,0.00,9887.48,12267.08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,333683659,33368365901,1,658800401.0,1479311,54,1,5,64,1,392,2016-08-02,10108.40,10108.40,1,K31,K31,D68,E87,E87,E87,I50,K20,K25,M06,N28,R10,R91,R11,R07,R19,01,435.95,435.95,6.0,1,None,9,1,01,1,1,0.0,0.0,0.0,0.00,10987.88,10987.88,0,0,0,0,0,

In [28]:
df['dx1'].astype

<bound method NDFrame.astype of 0         K56
1         J84
2         K92
3         R26
4         K31
         ... 
363293    N39
363294    D25
363295    N13
363296    A41
363297    F33
Name: dx1, Length: 363298, dtype: object>